In [2]:
import scrapy
from scrapy import Selector

import requests
import pandas as pd

In [25]:
listing_list = []

for page in range(0,136):
    url = 'https://www.smartshanghai.com/housing/apartments-rent/?page={}&bedrooms[0]=1&ownership_private=1&ownership_landlord=1&ownership_agency=1&view=list'.format(page)
    html = requests.get(url).text
    
    # Create a Selector selecting html as the HTML document
    sel = Selector(text = html)
    
    # Create a SelectorList of all div elements in the HTML document
    listings = sel.css('li.listing > a::attr(href)').extract()

    listing_list.extend(listings)

In [27]:
import time

appended_data = []

counter = 0

for listing in listing_list:
    try:
        html = requests.get(listing).text
        sel = Selector(text = html)
        
        apartment_dict = {}

        #1. ID
        apartment_dict['id'] = listing[-7:]

        #2. Description
        description = sel.xpath('//div[@class = "listing-info"]/h1/text()').extract()[0].strip()
        apartment_dict['description'] = description

        #3. Price
        price = sel.xpath('//div[@class = "price"]/span[@class = "price"]/text()').extract()[0].strip('')
        price = int(price.replace(',',''))

        apartment_dict['price'] = price

        #4. Currency
        currency = sel.xpath('//div[@class = "price"]/span[@class = "currency"]/text()').extract()[0].strip()

        apartment_dict['currency'] = currency

        #5. Period
        period = sel.xpath('//div[@class = "price"]/span[@class = "periodType"]/text()').extract()[0].strip()
    
        apartment_dict['period'] = period

        #6. Posted
        posted_time = sel.xpath('//div[@class = "listing-info"]/div[@class = "posted-and-views"]/span[1]/text()').extract()[0].strip()
        apartment_dict['posted_time'] = posted_time

        #7. Type
        property_type = sel.xpath('//div[@class = "details"]/ul/li[1]/div/text()').extract()[0].strip()
        apartment_dict['property_type'] = property_type
    
        #8. Commission
        commission = sel.xpath('//div[@class = "details"]/ul/li[3]/div/text()').extract()[0].strip()
        apartment_dict['comission'] = commission
    
        #9. Bedrooms
        bedrooms = sel.xpath('//span[@itemprop="numberOfBedrooms"]/text()').extract()[0].strip()
        apartment_dict['bedrooms'] = bedrooms

        #10. Bathrooms
        bathrooms = sel.xpath('//span[@itemprop="numberOfBathroomsTotal"]/text()').extract()[0].strip()
        apartment_dict['bathrooms'] = bathrooms

        #11. Size
        size = sel.xpath('//div[@class = "details"]/ul/li[5]/div/text()').extract()[0].strip()
        apartment_dict['size'] = size
    
        #12. Floor
        floor = sel.xpath('//div[@class = "details"]/ul/li[6]/div/text()').extract()[0].strip()
        apartment_dict['floor'] = floor
    
        #13. Furnished
        furnished = sel.xpath('//div[@class = "details"]/ul/li[7]/div/text()').extract()[0].strip()
        apartment_dict['furnished'] = furnished
    
        #14. Windows
        windows = sel.xpath('//div[@class = "details"]/ul/li[8]/div/text()').extract()[0].strip()
        apartment_dict['windows'] = windows
    
        #15. District
        district = sel.xpath('//div[@class = "details"]/ul/li[9]/div/text()').extract()[0].strip()
        apartment_dict['district'] = district
    
        #16. Area
        area = sel.xpath('//div[@class = "details"]/ul/li[10]/div/text()').extract()[0].strip()
        apartment_dict['area'] = area
    
        #17. Metro
        metro = sel.xpath('//div[@class = "details"]/ul/li[12]/div/a[1]/text()').extract()[0].strip()
        apartment_dict['metro'] = metro

        #18. Metro Proximity
        metro_proximity = sel.xpath('//div[@class = "details"]/ul/li[12]/div/text()').extract()[0].strip()
        apartment_dict['metro_proxim'] = metro_proximity

        #19. Listing source
        list_source = sel.xpath('//p[@class = "agency-title"]/text()').extract()[0].strip()
        apartment_dict['list_source'] = list_source

        appended_data.append(apartment_dict)

        counter = counter+1
    
        if counter%15 == 0:
            print('Aggregated listings: {}.'.format(counter), end = ' ')

        if counter%600 == 0:
            pd.DataFrame(appended_data).to_csv('/Users/ilya/Desktop/Computer-Science/Projects/Kepler/data/{}.csv'.format(str(listing[-7:])))
            print('Dataframe saved to local.', end = ' ')
            time.sleep(60)
    except:
        pass

Aggregated listings: 15. Aggregated listings: 30. Aggregated listings: 45. Aggregated listings: 60. Aggregated listings: 75. Aggregated listings: 90. Aggregated listings: 105. Aggregated listings: 120. Aggregated listings: 135. Aggregated listings: 150. Aggregated listings: 165. Aggregated listings: 180. Aggregated listings: 195. Aggregated listings: 210. Aggregated listings: 225. Aggregated listings: 240. Aggregated listings: 255. Aggregated listings: 270. Aggregated listings: 285. Aggregated listings: 300. Aggregated listings: 315. Aggregated listings: 330. Aggregated listings: 345. Aggregated listings: 360. Aggregated listings: 375. Aggregated listings: 390. Aggregated listings: 405. Aggregated listings: 420. Aggregated listings: 435. Aggregated listings: 450. Aggregated listings: 465. Aggregated listings: 480. Aggregated listings: 495. Aggregated listings: 510. Aggregated listings: 525. Aggregated listings: 540. Aggregated listings: 555. Aggregated listings: 570. Aggregated listing

In [28]:
main_df = pd.DataFrame(appended_data)

In [29]:
main_df

,id,description,price,currency,period,posted_time,property_type,comission,bedrooms,bathrooms,size,floor,furnished,windows,district,area,metro,metro_proxim,list_source
0,1447763,Bright 1Br+Yard @ L1/10/12 Shanxi rd station,8500,¥,/ Month,"August 18, 2021",Apartments,-,1,1,50 sqm,1,Unfurnished,South,Xuhui,-,South Shaanxi Rd,5 minutes walk to,Agent
1,1448399,"Elevator 1Br/1Lr/1Bt apt nr Jiangsu rd,Line2/3...",8000,¥,/ Month,"August 21, 2021, at 12:38 PM",Apartments,-,1,1,50 sqm,7,Furnished,South,Changning,-,Jiangsu Rd,6 minutes walk to,Agent
2,1448398,Jiaotong Uni. 1bed nice style historical street,15120,¥,/ Month,"August 21, 2021, at 12:18 PM",Apartments,35%,1,1,70 sqm,2,Furnished,South,Changning,-,Jiaotong University,10 minutes walk to,Agent
3,1448395,65sqm 1br Apartments/ChangPing Rd Area/Fully E...,12000,¥,/ Month,"August 21, 2021, at 11:42 AM",Apartments,35%,1,1,65 sqm,5,Furnished,South,Jing'an District,-,Wuning Rd,5 minutes walk to,Agent
4,1448394,Nice 1br Apartment @Nanjing West Road,9500,¥,/ Month,"August 21, 2021, at 11:35 AM",Apartments,35%,1,1,60 sqm,4,Furnished,South,Jing'an District,-,West Nanjing Rd,7 minutes walk to,Agent
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2033,1442833,One bedroom near jiangsu rd station,8500,¥,/ Month,"July 23, 2021",Apartments,35%,1,1,55 sqm,4,Furnished,South,Changning,Zhongshan Park,Jiangsu Rd,10 minutes walk to,Agent
2034,1442828,Jing'an Changping Rd. elevator 1bed,10500,¥,/ Month,"July 23, 2021",Apartments,35%,1,1,65 sqm,14,Furnished,South,Jing'an District,Jing'an Temple,Changping Rd,10 minutes walk to,Agent
2035,1442826,One bedroom near jiaotong university station,8000,¥,/ Month,"July 23, 2021",Apartments,35%,1,1,50 sqm,5,Furnished,South,Changning,-,Jiaotong University,10 minutes walk to,Agent
2036,1442823,4.5 m ceiling at Yongjia road only 8K@hengshan...,8000,¥,/ Month,"July 23, 2021",Apartments,-,1,1,60 sqm,3,Furnished,South,Huangpu,People's Square,Hengshan Rd,5 minutes walk to,Agent


In [30]:
pd.DataFrame(main_df).to_csv('/Users/ilya/Desktop/Computer-Science/Projects/Kepler/data/all_listings.csv', index = False)